In [1]:


import yaml
import sagemaker
import boto3
import pandas as pd
import os
import json
from time import gmtime, strftime
from sagemaker.sklearn.model import SKLearnModel
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:

import yaml
import sagemaker

SETTING_FILE_PATH = "../config/settings.yaml"

# AWSリソース設定読み込み
with open(SETTING_FILE_PATH) as file:
    aws_info = yaml.safe_load(file)

sess = sagemaker.Session()

role = aws_info["aws"]["sagemaker"]["role"]
bucket = aws_info["aws"]["sagemaker"]["s3bucket"]
region = aws_info["aws"]["sagemaker"]["region"]


In [3]:

! echo "this is dummy model" > dummy_model.txt |tar -czf dummy_model.tar.gz dummy_model.txt

In [4]:

prefix = "endpoint_latency_experiment"
model_file = "dummy_model.tar.gz"

s3_resource_session = boto3.Session().resource("s3").Bucket(bucket)
s3_resource_session.Object(os.path.join(prefix, "model", model_file)).upload_file(
    model_file
)

In [5]:
model_data = f"s3://{bucket}/{prefix}/model/{model_file}"
model = SKLearnModel(
    model_data=model_data,
    role=role,
    framework_version="0.23-1",
    py_version="py3",
    source_dir="model",
    entry_point="inference.py",
    sagemaker_session=sess,
)


In [11]:

model_name = "endpoint-latency-experiment-model"

sess.create_model(
    model_name, role, model.prepare_container_def()
)


'endpoint-latency-experiment-model'

In [34]:
def check_invoke_endopoint(endpoint_name: str):
    runtime = boto3.Session().client("sagemaker-runtime")
    response = runtime.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType="application/json",
        Accept="application/json",
        Body="0",
    )
    predictions = json.loads(response["Body"].read().decode("utf-8"))
    print("prediction: ",predictions)


In [35]:
def create_endpoint(instance_type: str):
    endpoint_name = "{}-{}".format("latency-experiment-endpoint", instance_type.replace(".", ""))
    predictor = model.deploy(
        initial_instance_count=1,
        instance_type=instance_type,
        endpoint_name=endpoint_name,
    )

    check_invoke_endopoint(endpoint_name)

In [36]:
# t2_instance_type_list = ["ml.t2.medium", "ml.t2.large", "ml.t2.xlarge", "ml.t2.2xlarge"]
t2_instance_type_list = ["ml.t2.large"]
m5_instance_type_list = ["ml.m5.large", "ml.m5.xlarge", "ml.m5.2xlarge", "ml.m5.4xlarge", "ml.m5.12xlarge", "ml.m5.24xlarge"]

In [32]:
for instance_type in t2_instance_type_list:
    print(instance_type)
    create_endpoint(instance_type=instance_type)

ml.t2.large
-------!prediction:  0
